In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import get_sun
from astropy.time import Time
from astropy.coordinates import FK5
from astropy.coordinates import SkyCoord

In [2]:
#Get declination and right ascension of the sun at a given time
time1 = Time('2022-9-25 12:00:00') #The time
sun = get_sun(time1)

sun_coord = SkyCoord(ra=sun.ra.degree*u.degree, dec=sun.dec.degree*u.degree, frame='icrs') #transform the sun coordinates to FK5
sun_coord.galactic

sun_coord_fk5 = sun_coord.transform_to(FK5(equinox='J2022.9')) #precess to a different equinox
sun_coord.transform_to(FK5(equinox='J2022.9'))

print("Copy Method: ", sun_coord_fk5.ra*u.degree, sun_coord_fk5.dec*u.degree)
print("Native Transform", sun_coord)

Copy Method:  182.21237755139268 deg2 -0.9585435276974851 deg2
Native Transform <SkyCoord (ICRS): (ra, dec) in deg
    (181.91889427, -0.83114161)>


In [ ]:
#formula to calculate the latitude of Ojai
# Latitude = arctan (cos(RA) * shadow_length / stick_length) * pi/180 + Dec

def latitude_calc(stick_length, shadow_length, sun_dec, sun_ra):
    latitude = np.arctan(np.cos(sun_ra) * shadow_length / stick_length) * (180/np.pi) + sun_dec
    return latitude


In [ ]:
#mean and standard deviation of the measured lengths of the shadow and stick 

#measurments of the shadow and stick
shadow_error = 0.1 #cm
shadow_lengths = [20.2, 21.1, 20.6, 20.5, 20.5, 20.9] #cm 
stick_error = 0.1 #cm
stick_lengths = [26.6, 26.7, 26.5, 26.6, 26.6, 26.5]

sun_ra = sun_coord.ra.degree #right ascension of the sun in degrees

sun_dec = sun_coord.dec.degree #declination of the sun in degrees

#calculate the latitude for each of these measuremnts 
latitudes = []
for i in range(len(shadow_lengths)):
    lat = latitude_calc(stick_lengths[i], shadow_lengths[i], sun_dec, sun_ra)
    print(stick_lengths[i], shadow_lengths[i], sun_dec, sun_ra)
    latitudes.append(lat)

print(latitudes)


In [ ]:
#Calculate the error propagation for the latitude
def calc_error(stick_length, stick_error, shadow_length, shadow_error, sun_dec, sun_ra):
    latitude = latitude_calc(stick_length, shadow_length, sun_dec, sun_ra)
    error = np.std(latitude)
    return error

#calculate the error for each of the measurements
errors = []
for i in range(len(shadow_lengths)):
    error = calc_error(stick_lengths[i], stick_error, shadow_lengths[i], shadow_error, sun_dec, sun_ra)
    errors.append(error)

print(errors)


plt.figure(1)
for i in range(len(latitudes)):
    plt.hist(np.random.normal(latitudes[i], errors[i], 1000000), bins=100, alpha=0.5, label='Measurement ' + str(i+1))

final = []
i = 0
for val in latitudes:
    if i == 0:
        final = np.random.normal(val,0.5,1000000)
    else:
        final += np.random.normal(val,0.5,1000000)
    i += 1

final /= np.float(len(latitudes))
plt.hist(final,bins=100,alpha=1,color='black',density=True)

plt.show()


In [ ]:
plt.figure(2)
sigma = 0.5
sz = 1000000
bins = 100
numbers = latitudes
mean = np.mean(latitudes)
n = len(latitudes)

plt.clf()
for val in numbers:
    plt.hist(np.random.normal(val,sigma,sz),bins=bins,alpha=0.75,density=True)

plt.xlabel('Actual Value')
plt.ylabel('Probability Density')
plt.title('distributions representing '+str(n)+' measurements')

# -- Take the mean of all these distributions
final = []
i = 0
for val in numbers:
    if i == 0:
        final = np.random.normal(val,sigma,sz)
    else:
        final += np.random.normal(val,sigma,sz)
    i += 1
    
final /= np.float(n)

plt.figure(4)
plt.hist(final,bins=bins,alpha=1,color='black',density=True)
plt.axvline(x=mean,linestyle='--',color='red')

print('The mean and standard deviation of the measurements: %.2f +/- %.2f'%(mean,sigma))
print('The mean and standard deviation of the mean of the measurements: %.2f +/- %.2f'% \
      (np.mean(final),np.std(final,ddof=1)))

In [ ]:
sz = 1000000
astropy_error = 0.00000001

sun_ra_dist = np.random.normal(sun_dec, astropy_error, sz)
sun_dec_dist = np.random.normal(sun_ra, astropy_error, sz)

plt.figure(3)
stick_lengths_dist = []
shadow_lengths_dist = []
for i in range(len(stick_lengths)):
    stick_lengths_dist.append(np.random.normal(stick_lengths[i], stick_error, sz))
    shadow_lengths_dist.append(np.random.normal(shadow_lengths[i], shadow_error, sz))
    plt.hist(stick_lengths_dist[i], bins=100, alpha=0.5, label='Stick Length ' + str(i+1))
    plt.hist(shadow_lengths_dist[i], bins=100, alpha=0.5, label='Shadow Length ' + str(i+1))

plt.show()
plt.figure(4)

latitudes_dist = []
for i in range(len(stick_lengths)):
    latitudes_dist.append(latitude_calc(stick_lengths_dist[i], shadow_lengths_dist[i], sun_dec_dist, sun_ra_dist))
    plt.hist(latitudes_dist[i], bins=100, alpha=0.5, label='Measurement ' )

plt.show()


In [ ]:
#Help from doc swift
a = np.random.normal(100, .1, 1000000)
b = np.random.normal(150, .5, 1000000)

plt.figure(1)
plt.hist(a, bins=200, density=True)
#plt.hist(b/1e6, bins=100)
#plt.hist((a+b)/1e6, bins=100)
#c = a+b
print(len(a), len(b), len(c))
plt.show()

plt.figure(2)
plt.hist(b, bins=200, density=True)
plt.show()

plt.figure(3)
plt.hist(a+b, bins=200, density=True)